In [5]:
 #获取3Q网站测试题(趣味图片题)
import requests
import re
from bs4 import BeautifulSoup

import json
import MySQLdb

from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
caps = webdriver.DesiredCapabilities.FIREFOX
import time
caps["marionette"]=True
binary = FirefoxBinary(r"D:\python外用包\firefox.exe")

headers={
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Mobile Safari/537.36'
}
link1='https://www.33iq.com/quiz/quwei'
link2='.html'
#循环遍历所有页
for i in range(1,4):
    if(i<=1):
        link=link1+link2
    else:
        link=link1+"/"+str(i)+link2
        
    #print(link)
    r = requests.get(link,headers=headers,timeout=10)
    r.encoding='gb2312'
    print("页响应状态码",r.status_code)
    
    #获取每个页面
    soup = BeautifulSoup(r.text,"lxml")
    
    #找到主要数据的div列表
    div_list = soup.find_all('div',class_='quiz-data')
    
    #遍历查询题目数量
    for eachdiv in div_list:
        i = re.findall('<div class="item".*?>(.*?)</div>',str(eachdiv))

    #判断题目数量是否为“1题”   
        if(str(i[0])=='1题'):
            title = eachdiv.a.text.strip()
            href = eachdiv.a.get('href')
            result="https://www.33iq.com/"+href
            linka="https://www.33iq.com/"+href.replace("quizresult","question")
        
            #题目编号获取
            #print(href[-8:-5])
            
            #爬取结果
            id=int(href[-8:-5])
            
            #提交的表单
            post_data = {
            'q_id':id,
            'qq_type':'2'
                        }
            
            headers2={
                 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
                    }
            r3 = requests.post(result,headers=headers2,timeout=10,data=post_data)
            r3.encoding='gb2312'
            soup2 = BeautifulSoup(r3.text,'lxml')
            tr_list = soup2.find('table')
            
            #print(tr_list)
            result=tr_list.find_all('p')
            Reault=[]
            for ea in result:
                each=str(ea).replace('<b class="score-orange font18">','').replace('</b>','').replace('<p>','').replace('</p>','')
                Reault.append(each)
            #print(Reault)
        
            
            
            #开始爬取题
            r2 = requests.get(linka,headers=headers,timeout=10)
            r2.encoding='gb2312'
            soup1 = BeautifulSoup(r2.text,"lxml")
            #print(soup1)
            
            #获取题目
            title_lxml = soup1.find('div',class_='controls word-wrap font13')
            title = title_lxml.p.text.strip()
            #print(title)
            
            #获取选项
            test_list = soup1.find_all('div',class_='answeroption btn btn-default btn-choose-quiz font13 btn-choose-single ')
            list_p=[]
            for i in range(0,len(test_list)):
                test=str(test_list[i]).replace('	','')
                p=test.replace(r'<div class="answeroption btn btn-default btn-choose-quiz font13 btn-choose-single ">','')
                t=p.replace('</div>','')
                list_p.append(t.strip())
                #print(t.strip())
            #print(list_p) 
            
            #获取图片
            img_list = soup1.find('img')
            if(str(img_list)=='None'):
                print("no img")

                shu={"Test_name": title,"question": {"Test_type": "单选题","question_title": title,"question_cont": list_p},"result": Reault}
                data = json.dumps(shu)
                print(data)
                conn  = MySQLdb.connect(host='localhost',user='root',passwd='linj',db='python')
                cur= conn.cursor()
                
                tsql = "insert into jsondata(id,type,format,data) values(c,a,b,'{json}')"
                sql = tsql.format(c=4,a='图片测试',b='单选',json=MySQLdb.escape_string(r'data'))
                cur.execute(sql)
                cur.close()
                conn.commit()
                conn.close()
                
                
                
            else:
                src=img_list.get('src')
                name=src[-25:-7]
                t = requests.get(src)
                with open('F:/About Me/img/'+name, 'wb') as f:
                    f.write(t.content)   
                path='F:/About Me/img/'+name
                #print(path)   
                shu={"Test_name": title,
                     "question": 
                     {"Test_type": "单选题",
                      "question_title": title,
                      "questtion_img": path,
                      "question_cont": list_p,
                     },
                      "result": Reault
                    }
               
                data= json.dumps(shu)
                
                conn  = MySQLdb.connect(host='localhost',user='root',passwd='linj',db='python')
                cur= conn.cursor()
                tsql = "insert into jsondata(id,type,format,data) values(c,a,b,'{json}')"
                sql = tsql.format(c=4,a='图片测试',b='单选',json=MySQLdb.escape_string(r'data'))
                
                cur.execute(sql)
              
                cur.close()
                conn.commit()
                conn.close()


        else:
            print("不符合要求",end=' ')

                    
    

页响应状态码 200
no img
{"Test_name": "\u4e00\u5217\u58eb\u5175\uff0c\u6bcf\u884c\u63927\u4e2a\uff0c\u4f594\u4e2a\uff1b\u6bcf\u884c\u639213\u4e2a\uff0c\u4f593\u4e2a\uff1b\u6bcf\u884c\u639217\u4e2a\uff0c\u4f592\u4e2a\uff1b\u95ee\u5217\u5175\u6709\u51e0\u4e2a\uff1f \u8bf7\u95ee\u4f60\u662f\u600e\u4e48\u8ba1\u7b97\u51fa\u7b54\u6848\u7684\u3002", "question": {"Test_type": "\u5355\u9009\u9898", "question_title": "\u4e00\u5217\u58eb\u5175\uff0c\u6bcf\u884c\u63927\u4e2a\uff0c\u4f594\u4e2a\uff1b\u6bcf\u884c\u639213\u4e2a\uff0c\u4f593\u4e2a\uff1b\u6bcf\u884c\u639217\u4e2a\uff0c\u4f592\u4e2a\uff1b\u95ee\u5217\u5175\u6709\u51e0\u4e2a\uff1f \u8bf7\u95ee\u4f60\u662f\u600e\u4e48\u8ba1\u7b97\u51fa\u7b54\u6848\u7684\u3002", "question_cont": ["1.\r\n\u7528\u679a\u4e3e\u6cd5\u5fc3\u7b97\uff0c\u7528\u65f6\u4e0d\u8d85\u8fc73\u5206\u949f", "2.\r\n\u5217\u51fa\u65b9\u7a0b\u6c42\u89e3\uff0c\u7528\u65f6\u4e0d\u8d85\u8fc75\u5206\u949f", "3.\r\n10\u5206\u949f\u5185\u6ca1\u5f97\u51fa\u7ed3\u8bba"]}, "result": ["\u6d4b

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'data'')' at line 1")

In [54]:
 for each in i:
            print(str(each))
           https://www.33iq.com/quiz/quwei

13题
5k+


In [25]:
a="7"
b=100*a
print(int(b)%8)

1


In [5]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
caps = webdriver.DesiredCapabilities.FIREFOX
import time

#表单数据
post_data = {
    'q_id':'575',
    'qq_type':'2'
}

caps["marionette"]=True
binary = FirefoxBinary(r"D:\python外用包\firefox.exe")

driver = webdriver.Firefox(firefox_binary=binary,capabilities=caps)
driver.get('https://www.33iq.com//quiz/quizresult/575.html',data=post_data)

houses = driver.find_elements_by_css_selector('table')
print(houses)

TypeError: get() got an unexpected keyword argument 'data'

In [6]:
import os
import requests
r = requests.get('https://a.33iq.com/upload/18/04/03/images/15227352204761.png!33.jpg')
with open('F:/About Me/img/img2.png', 'wb') as f:
    f.write(r.content)   
    
    post_data = {
    'entities_only':'true',
    'page':'1'
}
# 提交并获取返回数据
post_html = requests.post(url,data=post_data)
    

NameError: name 'url' is not defined

In [2]:
import requests
import re
from bs4 import BeautifulSoup

result='https://www.33iq.com//quiz/quizresult/575.html'
id=575
post_data = {
    'q_id':id,
    'qq_type':'2'
}
headers2={
                 'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
            }
r3 = requests.post(result,headers=headers2,timeout=10,data=post_data)
r3.encoding='gb2312'
soup2 = BeautifulSoup(r3.text,'lxml')
tr_list = soup2.find('table')
print(tr_list)
result=tr_list.find_all('p')
Reault=[]
for ea in result:
    each=str(ea).replace('<b class="score-orange font18">','').replace('</b>','').replace('<p>','').replace('</p>','')
    Reault.append(each)
print(Reault)
#print(soup2)

<table class="quiz-result">
<tbody>
<tr class="">
<td>
<p>测试得分： <b class="score-orange font18">0 </b> </p>
<p>结果分析：你的数学天赋还不成形，加油吧~</p>
</td>
</tr>
<tr class="hide">
<td>
<p>测试得分： <b class="score-orange font18">0 </b> </p>
<p>结果分析：你的数学天赋是欧拉型</p>
</td>
</tr>
<tr class="hide">
<td>
<p>测试得分： <b class="score-orange font18">0 </b> </p>
<p>结果分析：你的数学天赋是高斯型</p>
</td>
</tr>
</tbody>
</table>
['测试得分： 0  ', '结果分析：你的数学天赋还不成形，加油吧~', '测试得分： 0  ', '结果分析：你的数学天赋是欧拉型', '测试得分： 0  ', '结果分析：你的数学天赋是高斯型']


In [16]:
import json
import MySQLdb 


d={'name':'jack'}

#a=str(d)
#c=bytes(str(d), encoding = "utf8")
d_json=json.dumps('''d''')
c=bytes(d_json, encoding = "utf8")

conn = MySQLdb.connect(host='localhost', port=3306, user='root', passwd='linj', db='python',charset='utf8')
cur = conn.cursor()


tsql = "insert into test(data) values (%s)"%(c)
print(tsql)
#sql = tsql.format(a=MySQLdb.escape_string(c))

i=cur.execute(tsql)
   # Commit your changes in the database

# disconnect from server


print(i)


cur.execute('select data from test where id=17')
mydata = cur.fetchall()
print(mydata)
cur.close()
conn.commit()
conn.close()





insert into test(data) values (b'"d"')


ProgrammingError: (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near \'b\'"d"\')\' at line 1')